In [ ]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import nltk
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#importing input file
dff=pd.read_excel('Input.xlsx')

In [ ]:
dff

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


In [ ]:
url_list1 = []
for i in dff['URL']:
  url_list1.append(i)

# Data extraction

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_article_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Find and extract title
        title = soup.find('title').get_text() if soup.title else "No title found"

        # Find and extract main content
        main_content = soup.find('div', class_='td-post-content tagdiv-type')  # Customize based on the class name
        content = main_content.get_text() if main_content else "No content found"

        return title, content

    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            print(f"HTTP Error 404: Page not found for URL: {url}")
        else:
            print(f"HTTP Error {response.status_code}: {str(e)}")

        return None, None

# List of URLs to extract data from
url_list = url_list1

# Extract and store data
data = []
for url in url_list:
    title, content = extract_article_data(url)
    data.append({'Title': title, 'Content': content})

# Create a pandas DataFrame
df1 = pd.DataFrame(data)



HTTP Error 404: Page not found for URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
HTTP Error 404: Page not found for URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


In [ ]:
df = pd.concat([dff,df1],axis=1)

In [ ]:
df.head()

,URL_ID,URL,Title,Content
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\nTelemedicine, the use of technology to diagn..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,"\nThe rise of e-health, or the use of electron..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,No content found
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac..."


#EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   URL_ID   114 non-null    float64
 1   URL      114 non-null    object 
 2   Title    112 non-null    object 
 3   Content  112 non-null    object 
dtypes: float64(1), object(3)
memory usage: 3.7+ KB


In [ ]:
df.isnull().sum()

URL_ID     0
URL        0
Title      2
Content    2
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

URL_ID     0
URL        0
Title      0
Content    0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
# List of input file paths
input_files = ['StopWords_Names.txt', 'StopWords_Geographic.txt', 'StopWords_GenericLong.txt','StopWords_Generic.txt','StopWords_DatesandNumbers.txt','StopWords_Currencies.txt','StopWords_Auditor.txt']  # Replace with your file paths

# Output file path
output_file = 'combined_names.txt'

# Initialize an empty list to store names
all_names = []

# Read names from input files
for input_file in input_files:
    try:
        with open(input_file, 'r') as file:
            names = file.readlines()
            all_names.extend([name.strip() for name in names])
    except FileNotFoundError:
        print(f"File '{input_file}' not found.")
    except Exception as e:
        print("An error occurred:", str(e))

# Write combined names to output file
try:
    with open(output_file, 'w') as file:
        for name in all_names:
            file.write(name + '\n')
    print(f"Combined names written to '{output_file}'.")
except Exception as e:
    print("An error occurred:", str(e))


An error occurred: 'utf-8' codec can't decode byte 0xd3 in position 184: invalid continuation byte
Combined names written to 'combined_names.txt'.


In [ ]:
custom_stopwords  = all_names

Data Pre-Processing

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

# Custom stopwords list
custom_stopwords = custom_stopwords


def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenization
    tokens = [token for token in tokens if token.isalpha()]  # Remove punctuation and numbers

    # Remove both NLTK stopwords and custom stopwords
    stop_words = set(stopwords.words('english')) | set(custom_stopwords)
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords

    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]  # Stemming
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Apply text preprocessing to DataFrame
df['Preprocessed Text'] = df['Content'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Data visualization

##Positive score and Negative Score

In [ ]:
import pandas as pd
from textblob import TextBlob

def analyze_sentiment(text):
    blob = TextBlob(text)
    positive_score = 0
    negative_score = 0

    for sentence in blob.sentences:
        sentence_sentiment = sentence.sentiment.polarity
        if sentence_sentiment > 0:
            positive_score += sentence_sentiment
        elif sentence_sentiment < 0:
            negative_score += sentence_sentiment

    return positive_score, negative_score

# Apply sentiment analysis to DataFrame
df['Positive Score'], df['Negative Score'] = zip(*df['Preprocessed Text'].apply(analyze_sentiment))


# Polarity score

In [ ]:
import pandas as pd
from textblob import TextBlob

def calculate_polarity(positive_score,negative_score):
    PolarityScore = (positive_score-negative_score)/ ((positive_score + negative_score) + 0.000001)
    return PolarityScore

# Apply polarity analysis to DataFrame
df['Polarity_Score'] = calculate_polarity(df['Positive Score'],df['Negative Score'])

df['Polarity_Score']

0      0.999978
1     -1.000016
2      0.000000
3      0.999973
4      0.999973
         ...   
109   -1.000008
110   -1.000048
111    0.999963
112    0.999962
113   -1.000991
Name: Polarity_Score, Length: 112, dtype: float64

##subjective score

In [ ]:
import pandas as pd
from textblob import TextBlob

def calculate_Subjectivity(positive_score,negative_score,sentences):
  total_words = sum(len(sentence.split()) for sentence in sentences)

  Subjectivity_Score = (positive_score + negative_score)/ ((total_words) + 0.000001)
  return Subjectivity_Score

# Apply polarity analysis to DataFrame
df['Subjectivity_Score'] = calculate_Subjectivity(df['Positive Score'],df['Negative Score'],df['Preprocessed Text'])


##avarage santence lengh

In [ ]:
import pandas as pd
import re

def calculate_avg_sentence_length(text):
    sentences = re.split(r'[.!?]', text)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    num_sentences = len(sentences)

    total_words = sum(len(sentence.split()) for sentence in sentences)

    if num_sentences == 0:
        return 0
    else:
        return total_words / num_sentences

# Apply average sentence length analysis to DataFrame
df['Avg Sentence Length'] = df['Preprocessed Text'].apply(calculate_avg_sentence_length)


##PERCENTAGE OF COMPLEX WORDS

In [21]:
#importing tokenize library
from nltk.tokenize import word_tokenize

In [24]:
txt_list=[]
length=df.shape[0]
for i in range(0,length):
  txt=' '.join([word for word in df['Preprocessed Text']])
  txt_list.append(txt)



In [25]:
#tokenization of text
tokenize_text=[]
for i in txt_list:

  tokenize_text+=(word_tokenize(i))

In [33]:

vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
  if(count>2):
   complex_Word_Count+=1
  count=0

df['complex words count'] = complex_Word_Count

In [ ]:
df.info()

##FOG INDEX

In [34]:
# Calculate Fog Index for DataFrame
df['Fog Index'] = 0.4 * (df['Avg Sentence Length'] + df['complex words count'])


##AVG NUMBER OF WORDS PER SENTENCE

In [35]:
import pandas as pd
import nltk
nltk.download('punkt')

def calculate_avg_words_per_sentence(text):
    sentences = nltk.sent_tokenize(text)  # Tokenize into sentences
    num_sentences = len(sentences)

    words = nltk.word_tokenize(text)  # Tokenize into words
    num_words = len(words)

    if num_sentences == 0:
        return 0
    else:
        avg_words_per_sentence = num_words / num_sentences
        return avg_words_per_sentence

# Apply average words per sentence calculation to DataFrame
df['Average Words Per Sentence'] = df['Preprocessed Text'].apply(calculate_avg_words_per_sentence)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##COMPLEX WORD COUNT

In [36]:
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
  if(count>2):
   complex_Word_Count+=1
  count=0
print('complex words count=',  complex_Word_Count)


complex words count= 1672832


##WORD COUNT

In [37]:
import pandas as pd
import nltk
nltk.download('punkt')

def calculate_word_count(text):
    words = nltk.word_tokenize(text)  # Tokenize into words
    return len(words)

# Apply word count calculation to DataFrame
df['Word Count'] = df['Preprocessed Text'].apply(calculate_word_count)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##SYLLABLE PER WORD

In [39]:
vowels=['a','e','i','o','u']
import re
count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
syllable_count=count
df['syllable_per_word']=syllable_count

##PERSONAL PRONOUNS

In [40]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def count_personal_pronouns(text):
    words = nltk.word_tokenize(text)  # Tokenize into words
    tagged_words = nltk.pos_tag(words)  # Part-of-speech tagging

    personal_pronoun_count = sum(1 for word, pos in tagged_words if pos == 'PRP')
    return personal_pronoun_count

# Apply personal pronoun count calculation to DataFrame
df['Personal Pronoun Count'] = df['Preprocessed Text'].apply(count_personal_pronouns)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [41]:
df['Personal Pronoun Count'].unique()

array([0, 1])

##AVG WORD LENGTH

In [42]:
import pandas as pd
import nltk
nltk.download('punkt')

def calculate_avg_word_length(text):
    words = nltk.word_tokenize(text)  # Tokenize into words
    total_characters = sum(len(word) for word in words)
    total_words = len(words)

    if total_words == 0:
        return 0
    else:
        avg_word_length = total_characters / total_words
        return avg_word_length

# Apply average word length calculation to DataFrame
df['Average Word Length'] = df['Preprocessed Text'].apply(calculate_avg_word_length)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
df

,URL_ID,URL,Title,Content,Preprocessed Text,Positive Score,Negative Score,Polarity_Score,Subjectivity_Score,Avg Sentence Length,complex words count,Fog Index,Average Words Per Sentence,Word Count,syllable_per_word,Personal Pronoun Count,Average Word Length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\nTelemedicine, the use of technology to diagn...",telemedicin technolog diagnos treat patient re...,0.045950,0.000000,0.999978,8.124996e-07,841.0,1672832,669469.2,841.0,841,13651680,0,6.403092
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,"\nThe rise of e-health, or the use of electron...",rise electron mean facilit health care major d...,0.000000,-0.061364,-1.000016,-1.085045e-06,269.0,1672832,669240.4,269.0,269,13651680,0,6.546468
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,No content found,content found,0.000000,0.000000,0.000000,0.000000e+00,2.0,1672832,669133.6,2.0,2,13651680,0,6.000000
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac...",gain qualiti afford access mean world evolv ad...,0.036410,0.000000,0.999973,6.438007e-07,614.0,1672832,669378.4,614.0,614,13651680,0,6.135179
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac...",gain qualiti afford access mean world evolv ad...,0.036410,0.000000,0.999973,6.438007e-07,614.0,1672832,669378.4,614.0,614,13651680,0,6.135179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus: Impact on the Hospitality Industr...,\nBefore jumping on the topic I would like to ...,jump topic give overview coronaviru spread sym...,0.000000,-0.127667,-1.000008,-2.257429e-06,348.0,1672832,669272.0,348.0,348,13651680,0,5.666667
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus impact on energy markets | Blackco...,\nAs the coronavirus spreads around the world ...,coronaviru spread world countri implement cont...,0.000000,-0.020675,-1.000048,-3.655817e-07,943.0,1672832,669510.0,943.0,943,13651680,0,5.667020
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,What are the key policies that will mitigate t...,"\nFrom Alibaba to Ping An and Google to Ford, ...",alibaba ping googl ford compani globe tell sta...,0.027257,0.000000,0.999963,4.819572e-07,856.0,1672832,669475.2,856.0,856,13651680,0,5.627336
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,Marketing Drives Results With A Focus On Probl...,"\nWhen\nthe British ruled India, many Indians\...",british rule india indian accept work polici r...,0.026615,0.000000,0.999962,4.706048e-07,597.0,1672832,669371.6,597.0,597,13651680,0,5.668342


In [43]:
df.to_csv('Output Data Structure.csv', index=False)